# Week 2 - Preprocessing, part 2

# 1. Lesson: None

# 2. Weekly graph question

The Storytelling With Data book mentions planning on a "Who, What, and How" for your data story.  Write down a possible Who, What, and How for your data, using the ideas in the book.

In [ ]:
# Who, What, and How Summary for Manufacturing Defects Dataset

# Who: The primary audience is the quality control team and manufacturing managers, but may include other audiences such as
# various levels of decision-makers/leadership in order to make the proper investments to subvert costs or increase revenew.
# They need insights into defect trends to improve production efficiency and reduce waste.

# What: The key takeaway is to identify the most common manufacturing defects,
# analyze defect rates over time, and determine any patterns related to specific production lines or shifts.
# This will help managers implement targeted quality control measures.

# How: The best way to present this information is through:
# - Bar charts to show defect frequency by type.
# - Line graphs to track defect rates over time.
# - Heatmaps to highlight high-defect areas in different production stages.
# - Scatter plots to visualize defect types by production line or shift.


# 3. Homework - work with your own data

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import re
import random
import io
import zipfile
from datetime import datetime, timedelta

OSError: Could not find kaggle.json. Make sure it's located in /home/codespace/.config/kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/

This week, you will do the same types of exercises as last week, but you should use your own datasets that you found last semester.

### Here are some types of analysis you can do  Use Google, documentation, and ChatGPT to help you:

- Summarize the datasets using info() and describe()

- Are there any duplicate rows?

- Are there any duplicate values in a given column (when this would be inappropriate?)

- What are the mean, median, and mode of each column?

- Are there any missing or null values?

    - Do you want to fill in the missing value with a mean value?  A value of your choice?  Remove that row?

- Identify any other inconsistent data (e.g. someone seems to be taking an action before they are born.)

- Encode any categorical variables (e.g. with one-hot encoding.)

### Conclusions:

- Are the data usable?  If not, find some new data!

- Do you need to modify or correct the data in some way?

- Is there any class imbalance?  (Categories that have many more items than other categories).

### 3a. Data Collection, Pre-Processing, Processing, & Mining

In [3]:
from kaggle.api.kaggle_api_extended import KaggleApi
import pandas as pd

# Initialize and authenticate the API
api = KaggleApi()
api.authenticate()

# Define the dataset and file name
dataset = 'fahmidachowdhury/manufacturing-defects'
file_name = 'Manufacturing_Defects.csv'  # Update with the correct file name if different

# Download the dataset file
api.dataset_download_file(dataset, file_name, path='.')

# Load the dataset into a pandas DataFrame
df = pd.read_csv(file_name)

# Display the first few rows of the DataFrame
print(df.head())

OSError: Could not find kaggle.json. Make sure it's located in /home/codespace/.config/kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/

# 4. Storytelling With Data graph

Just like last week: choose any graph in the Introduction of Storytelling With Data. Use matplotlib to reproduce it in a rough way. I don't expect you to spend an enormous amount of time on this; I understand that you likely will not have time to re-create every feature of the graph. However, if you're excited about learning to use matplotlib, this is a good way to do that. You don't have to duplicate the exact values on the graph; just the same rough shape will be enough.  If you don't feel comfortable using matplotlib yet, do the best you can and write down what you tried or what Google searches you did to find the answers.